In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import AUPSNet
from model import u2net

In [2]:
# normalize the predicted SOD probability map
def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

In [3]:
def save_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split(os.sep)[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)
    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    imo.save(d_dir+imidx+'.png')

In [4]:
def main():

    # --------- 1. get image path and name ---------
    image_dir = "In-kind_shooting/src/"
    prediction_dir = "In-kind_shooting/res/"
    model_dir = "/mnt/saved_models/U2Net/u2net_bce_itr_21750_train_0.227544_tar_0.017907.pth"
    img_name_list = glob.glob(image_dir + os.sep + '*')
    print(img_name_list)

    # --------- 2. dataloader ---------
    #1. dataloader
    test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                        lbl_name_list = [],
                                        transform=transforms.Compose([RescaleT(320),
                                                                      ToTensorLab(flag=0)])
                                        )
    test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                        batch_size=1,
                                        shuffle=False,
                                        num_workers=1)

    # --------- 3. model define ---------

    net = u2net.U2NET(3,1)
    #net = nn.DataParallel(net) # multi-GPU
    net.load_state_dict(torch.load(model_dir))
    if torch.cuda.is_available():
        net.cuda()
    net.eval()

    # --------- 4. inference for each image ---------
    for i_test, data_test in enumerate(test_salobj_dataloader):

        print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

        inputs_test = data_test['image']
        inputs_test = inputs_test.type(torch.FloatTensor)

        if torch.cuda.is_available():
            inputs_test = Variable(inputs_test.cuda())
        else:
            inputs_test = Variable(inputs_test)

        d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

        # normalization
        pred = d1[:,0,:,:]
        pred = normPRED(pred)

        # save results to test_results folder
        if not os.path.exists(prediction_dir):
            os.makedirs(prediction_dir, exist_ok=True)
        save_output(img_name_list[i_test],pred,prediction_dir)

        del d1,d2,d3,d4,d5,d6,d7

In [7]:
if __name__ == "__main__":
    main()

['In-kind_shooting/src/1.jpg', 'In-kind_shooting/src/2.jpg', 'In-kind_shooting/src/3.jpg', 'In-kind_shooting/src/4.jpg', 'In-kind_shooting/src/5.jpg', 'In-kind_shooting/src/6.jpg', 'In-kind_shooting/src/7.jpg', 'In-kind_shooting/src/8.jpg', 'In-kind_shooting/src/9.jpg', 'In-kind_shooting/src/10.jpg', 'In-kind_shooting/src/14.jpg', 'In-kind_shooting/src/15.jpg', 'In-kind_shooting/src/12.jpg', 'In-kind_shooting/src/13.jpg', 'In-kind_shooting/src/11.jpg']
inferencing: 1.jpg
inferencing: 2.jpg
inferencing: 3.jpg
inferencing: 4.jpg
inferencing: 5.jpg
inferencing: 6.jpg
inferencing: 7.jpg
inferencing: 8.jpg
inferencing: 9.jpg
inferencing: 10.jpg
inferencing: 14.jpg
inferencing: 15.jpg
inferencing: 12.jpg
inferencing: 13.jpg
inferencing: 11.jpg
